In [ ]:
!pip install pretrainedmodels

In [ ]:
import numpy as np 
import pandas as pd
import tqdm
import pathlib
import shutil
import os 
import torch
import seaborn as sns
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import gc

from PIL import Image
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict
from torch import optim
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import ReduceLROnPlateau

import pretrainedmodels


In [ ]:
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(721)

In [ ]:
train_data = "../input/cassava-leaf-disease-classification/train_images"
train_df = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
train_data_2019 = "../input/train-images-augumented/train_2019"

In [ ]:
train_df.head()

In [ ]:
## split into folds
train_df_vs = train_df.copy()
s_fold = StratifiedKFold(n_splits=5, shuffle=True)
for n, (train_index, val_index) in enumerate(s_fold.split(train_df_vs, train_df_vs['label'])):
    train_df_vs.loc[val_index, 'fold'] = int(n)
train_df_vs['fold'] = train_df_vs['fold'].astype(int)
print(train_df_vs.groupby(['fold', 'label']).size()) 


In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, df, root, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.labels = df['label'].values
        self.transform = transform
        self.root = root
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        file_name = self.file_names[index]
        file_path = f'{self.root}/{file_name}'
#         image = Image.open(file_path)
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
       
        if self.transform:
            image = self.transform(image=image)["image"]
        label = torch.tensor(self.labels[index]).long()
        return image, label
        
    

In [ ]:
#transformations
# train_transforms = transforms.Compose([
#     transforms.Resize((512, 512)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
# ])
# val_transforms = transforms.Compose([
#     transforms.Resize((512, 512)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
# ])


from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
            CenterCrop(512, 512, p=1.),
            Resize(512, 512),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms():
    return Compose([
            CenterCrop(512, 512, p=1.),
            Resize(512, 512),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

# def get_inference_transforms():
#     return Compose([
#             RandomResizedCrop(512, 512),
#             Transpose(p=0.5),
#             HorizontalFlip(p=0.5),
#             VerticalFlip(p=0.5),
#             HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
#             RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
#             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
#             ToTensorV2(p=1.0),
#         ], p=1.)

In [ ]:
# model = models.resnext50_32x4d(pretrained=True)

# for param in model.parameters():
#     param.required_grad = False

In [ ]:
# from collections import OrderedDict

# classifier = nn.Sequential(OrderedDict([
#               ('fc1', nn.Linear(2048, 126)),
#               ('relu1', nn.ReLU()),
#               ('dropout1', nn.Dropout(p=0.3)),
# #               ('fc2', nn.Linear(256, 126)),
# #               ('relu2', nn.ReLU()),
# #               ('dropout2', nn.Dropout(p=0.2)),
#               ('fc3', nn.Linear(126, 5)),
             
              
# ]))

# model.fc = classifier

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# criterion = nn.CrossEntropyLoss()

# plist = [{'params': model.parameters(), 'lr': 0.00003}]
# optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-6)

# optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)



In [ ]:
class TaylorSoftmax(nn.Module):

    def __init__(self, dim=1, n=2):
        super(TaylorSoftmax, self).__init__()
        assert n % 2 == 0
        self.dim = dim
        self.n = n

    def forward(self, x):
        
        fn = torch.ones_like(x)
        denor = 1.
        for i in range(1, self.n+1):
            denor *= i
            fn = fn + x.pow(i) / denor
        out = fn / fn.sum(dim=self.dim, keepdims=True)
        return out

class LabelSmoothingLoss(nn.Module):

    def __init__(self, classes, smoothing=0.0, dim=-1): 
        super(LabelSmoothingLoss, self).__init__() 
        self.confidence = 1.0 - smoothing 
        self.smoothing = smoothing 
        self.cls = classes 
        self.dim = dim 
    def forward(self, pred, target): 
        """Taylor Softmax and log are already applied on the logits"""
        #pred = pred.log_softmax(dim=self.dim) 
        with torch.no_grad(): 
            true_dist = torch.zeros_like(pred) 
            true_dist.fill_(self.smoothing / (self.cls - 1)) 
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence) 
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))
    

class TaylorCrossEntropyLoss(nn.Module):

    def __init__(self, n=2, ignore_index=-1, reduction='mean', smoothing=0.2):
        super(TaylorCrossEntropyLoss, self).__init__()
        assert n % 2 == 0
        self.taylor_softmax = TaylorSoftmax(dim=1, n=n)
        self.reduction = reduction
        self.ignore_index = ignore_index
        self.lab_smooth = LabelSmoothingLoss(5, smoothing=smoothing)

    def forward(self, logits, labels):

        log_probs = self.taylor_softmax(logits).log()
        #loss = F.nll_loss(log_probs, labels, reduction=self.reduction,
        #        ignore_index=self.ignore_index)
        loss = self.lab_smooth(log_probs, labels)
        return loss


In [ ]:
        
criterion = TaylorCrossEntropyLoss(n=2, smoothing=0.3)

In [ ]:
def se_resnext50_32x4d(pretrained=False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.se_resnext50_32x4d(pretrained=pretrained)
    return model


In [ ]:
train_2019 = pd.read_csv("../input/train-imagaes-aug/train_2019.csv")

In [ ]:
train_2019 = train_2019.drop(["Unnamed: 0"], axis=1)

In [ ]:
epochs =6
folds = StratifiedKFold(n_splits=5, shuffle=True, ).split(np.arange(train_df.shape[0]), train_df.label.values)
for fold, (trn_idx, val_idx) in enumerate(folds):
    ### initate model here so we can del the weights later
   
 #     model = models.resnext50_32x4d(pretrained=True)

#     for param in model.parameters():
#         param.required_grad = False

#     classifier = nn.Sequential(OrderedDict([
#               ('fc1', nn.Linear(2048, 126)),
#               ('relu1', nn.ReLU()),
#              ('dropout1', nn.Dropout(p=0.2)),

#              ('fc3', nn.Linear(126, 5)),


#                 ]))

#     model.fc = classifier
    model = se_resnext50_32x4d(pretrained=True)
    
    for param in model.parameters():
        param.required_grad = False
    model.avg_pool = nn.AdaptiveAvgPool2d(1)
    model.dropout= nn.Dropout(0.3)
    num_ftrs = model.last_linear.in_features
    
    model.last_linear = nn.Linear(num_ftrs, 5)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-6, amsgrad=True)
    
    model.to(device)
    
    model.class_to_idx = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}
    
    print(f"========== fold: {fold} training ==========")
    
    
    train_ = train_df.loc[trn_idx,:].reset_index(drop=True)
    valid_ = train_df.loc[val_idx,:].reset_index(drop=True)
    
   
    train_dataset = CassavaDataset(train_, train_data,
                                     transform=get_train_transforms())
    
    valid_dataset = CassavaDataset(valid_, train_data,
                                     transform=get_valid_transforms())
  
    train_loader = DataLoader(train_dataset, 
                                  batch_size=16, 
                                  shuffle=True, 
                                  num_workers=4)
    val_loader = DataLoader(valid_dataset, 
                                  batch_size=16, 
                                  shuffle=False, 
                                  num_workers=4)

        
        
    for epoch in range(epochs):
        print(f"========== epoch: {epoch} training ==========")
        running_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        else:
            validation_loss = 0
            accuracy = 0

            with torch.no_grad():
                model.eval()
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    validation_loss += loss.item()


                    ps = torch.exp(outputs)

                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))

            model.train()

            print("Epoch: {}/{}.. ".format(epoch+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
                  "Valid Loss: {:.3f}.. ".format(validation_loss/len(val_loader)),
              "Valid Accuracy: {:.3f}".format(accuracy/len(val_loader)))
            torch.save({"state_dict":model.state_dict(),
           "class_to_idx":model.class_to_idx}, '{}_rese_cassava_classifier_fold_{}.pth'.format(epoch+1, fold))
    del model
    gc.collect()


In [ ]:
# 0 = 66
# 1 = 385
# 2 = 392
# 3 = 6210
# 4 = 45

    "0":string"Cassava Bacterial Blight (CBB)"  - 1087
    "1":string"Cassava Brown Streak Disease (CBSD)" - 2189
    "2":string"Cassava Green Mottle (CGM)" - 2386
    "3":string"Cassava Mosaic Disease (CMD)" - 13158
    "4":string"Healthy" - 2577
